In [1]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print("length: ", len(text))

length:  1115394


In [2]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [4]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [5]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [6]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [7]:
block_size = 8 #context size
train_data[:block_size+1:1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [8]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when intput is {context} the target is: {target}")

when intput is tensor([18]) the target is: 47
when intput is tensor([18, 47]) the target is: 56
when intput is tensor([18, 47, 56]) the target is: 57
when intput is tensor([18, 47, 56, 57]) the target is: 58
when intput is tensor([18, 47, 56, 57, 58]) the target is: 1
when intput is tensor([18, 47, 56, 57, 58,  1]) the target is: 15
when intput is tensor([18, 47, 56, 57, 58,  1, 15]) the target is: 47
when intput is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is: 58


In [9]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) #won't we get outside array?
    return x,y
xb, yb = get_batch('train')
print("inputs")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)



inputs
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [10]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    def forward(self, idx, targets = None):
        logits = self.token_embedding_table(idx) # B, T, C
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            #get predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # from B T C it makes B -1 C
            # apply softmax to get probablities
            probs = F.softmax(logits, dim=-1) # B,C
            # sample from next distribution
            idx_next = torch.multinomial(probs, num_samples=1) # B, 1
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

#initial loss should be -ln(1/65), where 65 is the vocabulary size

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [11]:
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [12]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3) #usually should be 1e-4 but for such simple model it's enough

In [13]:
batch_size = 32
for steps in range(10000):
    #sample
    xb, yb = get_batch('train')
    #evaluate
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.5727508068084717


In [14]:
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y helti


In [15]:
B, T, C = 4,8,2
x = torch.randn(B,T,C)
x.shape


torch.Size([4, 8, 2])

In [17]:
xbow = torch.zeros((B, T, C)) #bow = bag of words
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # t,C
        xbow[b,t] = torch.mean(xprev, 0)

In [18]:
x[0]

tensor([[-3.5685e-01, -1.4841e+00],
        [ 1.4718e-03, -9.1119e-01],
        [-4.2776e-01, -1.4516e+00],
        [ 5.2601e-01, -2.5594e-02],
        [ 4.3046e-01, -9.9963e-01],
        [-6.9346e-01,  9.3705e-01],
        [-1.1372e+00, -1.1862e+00],
        [-9.0355e-01, -1.1036e+00]])

In [19]:
xbow[0]

tensor([[-0.3568, -1.4841],
        [-0.1777, -1.1976],
        [-0.2610, -1.2823],
        [-0.0643, -0.9681],
        [ 0.0347, -0.9744],
        [-0.0867, -0.6558],
        [-0.2368, -0.7316],
        [-0.3201, -0.7781]])